In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-03-06 03:31:07,247 INFO: Initializing external client
2025-03-06 03:31:07,247 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:31:07,991 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Fetching data from 2025-02-05 08:31:07.246359+00:00 to 2025-03-06 07:31:07.246359+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.23s) 


In [5]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-05 09:00:00
1,0,0,0,1,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,3,2025-03-05 09:00:00
2,2,3,2,0,1,3,2,1,1,3,...,3,3,0,0,0,1,8,15,4,2025-03-05 09:00:00
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-05 09:00:00
4,2,1,4,1,2,1,2,2,2,1,...,0,2,0,2,0,2,1,7,7,2025-03-05 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,259,2025-03-05 09:00:00
248,1,0,3,2,5,0,1,2,0,0,...,0,0,0,1,2,0,0,3,260,2025-03-05 09:00:00
249,14,12,23,23,25,28,23,29,40,60,...,1,0,0,0,2,0,4,15,261,2025-03-05 09:00:00
250,158,114,84,84,65,73,99,89,115,107,...,2,2,4,7,18,79,171,196,262,2025-03-05 09:00:00


In [6]:
predictions = fetch_next_hour_predictions()

2025-03-06 03:31:17,874 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 03:31:17,875 INFO: Initializing external client
2025-03-06 03:31:17,875 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:31:18,507 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.37s) 
Current UTC time: 2025-03-06 08:31:17.874209+00:00
Next hour: 2025-03-06 09:00:00+00:00
Found 252 records


In [7]:
predictions


,pickup_location_id,predicted_demand,pickup_hour
252,52,1.0,2025-03-06 09:00:00+00:00
253,210,0.0,2025-03-06 09:00:00+00:00
254,111,0.0,2025-03-06 09:00:00+00:00
255,80,3.0,2025-03-06 09:00:00+00:00
256,23,0.0,2025-03-06 09:00:00+00:00
...,...,...,...
499,151,55.0,2025-03-06 09:00:00+00:00
500,36,0.0,2025-03-06 09:00:00+00:00
501,50,38.0,2025-03-06 09:00:00+00:00
502,185,0.0,2025-03-06 09:00:00+00:00


In [8]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [9]:
top10

[161, 237, 162, 138, 142, 48, 239, 236, 170, 230]

In [10]:
top10

[161, 237, 162, 138, 142, 48, 239, 236, 170, 230]

In [11]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
247    False
248    False
249    False
250    False
251    False
Name: pickup_location_id, Length: 252, dtype: bool

In [12]:
features_cp = features.copy()

In [13]:
from datetime import timedelta
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]
time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]
time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": time_series_values}
)

In [14]:
historical_df

,datetime,rides
0,2025-02-05 09:00:00,214.0
1,2025-02-05 10:00:00,243.0
2,2025-02-05 11:00:00,234.0
3,2025-02-05 12:00:00,310.0
4,2025-02-05 13:00:00,342.0
...,...,...
668,2025-03-05 05:00:00,15.0
669,2025-03-05 06:00:00,40.0
670,2025-03-05 07:00:00,123.0
671,2025-03-05 08:00:00,146.0
